In [109]:
from os.path import join as pathJoin
import pandas as pd
import numpy as np
import pickle
import scipy.spatial.distance as dist
from sklearn.preprocessing import normalize

In [2]:
# data_root = "/media/rauf/TOSHIBA EXT/STORAGE/DATA/movies/tag-genome/tag-genome"
data_root = "/home/rauf/Programs/data/movies"

MovieID_TagID_Relevance = pathJoin(data_root, "tag_relevance.dat")
MovieID_Title_MoviePopularity = pathJoin(data_root, "movies.dat")
TagID_Tag_TagPopularity = pathJoin(data_root, "tags.dat")

In [3]:
tag_relevance = pd.read_csv(MovieID_TagID_Relevance, delimiter='\t', header=None, names=['MovieID', 'TagID', 'Relevance'])
movies = pd.read_csv(MovieID_Title_MoviePopularity, delimiter='\t', header=None, names=['MovieID', 'Title', 'MoviePopularity'])
tags = pd.read_csv(MovieID_Title_MoviePopularity, delimiter='\t', header=None, names=['TagID', 'Tag', 'TagPopularity'])

In [4]:
with open('pickled/genome.pickle', 'rb') as f:
    genome = pickle.load(f)

In [19]:
genome = np.array(genome)

In [49]:
genome = genome.T

In [50]:
def popularity(tagId):
    return tags.TagPopularity[tagId]

In [51]:
def docFreq(tag):
    byTag = genome[:, tag]
    specific = [x for x in byTag if x > 0.5]
    return len(specific)

In [52]:
nTags = tags.shape[0]
nTags

9734

In [53]:
# rows - movies
# columns - tags
genome.shape

(1128, 9734)

In [115]:
tag_weights = np.array([np.log(popularity(tagId)) / np.log(docFreq(tagId)) for tagId in range(nTags)])

In [142]:
# need to get rid of inf
sorted(tag_weights, reverse=True)[:5]

[inf,
 8.1344263202209266,
 5.2702488681577071,
 5.1978546720042988,
 4.7598181264216066]

In [145]:
infID = np.argmax(tag_weights)
tag_weights[infID] = 9

In [162]:
def weightedCosSimi(u, v):
    '''Not exactly like in the article'''
    return dist.cosine(u * tag_weights, v * tag_weights)

In [167]:
def articleCosSimi(u, v):
    '''Exactly like in the article'''
    w = tag_weights
    x = np.sum(w * u * v)
    y = np.sqrt(sum(w * u * u)) * np.sqrt(sum(w * v * v)) 
    return  x / y 

In [169]:
# lets compare
# we can just try both
print(weightedCosSimi(u, v))
print(articleCosSimi(u, v))

0.142785262082
0.848177542814


In [170]:
def rel(tagId, movId):
    return genome[movId, tagId]

In [172]:
def critiqueDist(critiquedMovieId, retrievedMovieId, tagId, direction):
    ic, ir, t, d  = critiquedMovieId, retrievedMovieId, tagId, direction
    return max(0, rel(t, ir) - rel(t, ic) * d)

In [ ]:
# TODO
# neighbors
# N_+ N_- N_0
# critique-entropy